In [ ]:
@njit
def v_1(w, model):
    """
    Computes lifetime value at t=1 given current wage w_1 = w
    """
    β, c = model.β, model.c
    s = np.maximum(c, model.w_vals)
    h_1 = c + β * np.sum(s * model.φ)
    return np.maximum(w + β * w, h_1)


@njit
def res_wage(model):
    """
    Computes reservation wage at t=1
    """
    β, c = model.β, model.c
    s = np.maximum(c, model.w_vals)
    h_1 = c + β * np.sum(s * model.φ)
    return h_1 / (1 + β)



##### Plots #####

default_model = create_job_search_model()


def fig_dist(model=default_model, fs=10):
    """
    Plot the distribution of wages
    """
    fig, ax = plt.subplots()
    ax.plot(model.w_vals, model.φ, "-o", alpha=0.5, label="wage distribution")
    ax.legend(loc="upper left", fontsize=fs)
    plt.show()


def fig_v1(model=default_model, savefig=False,
           figname="../figures_py/iid_job_search_0_py.png", fs=12):
    """
    Plot two-period value function and res wage
    """
    n, w_vals, φ, β, c = model

    v = [v_1(w, model) for w in w_vals]
    w_star = res_wage(model)
    s = np.maximum(c, w_vals)
    continuation_val = c + β * np.sum(s * φ)
    min_w, max_w = np.min(w_vals), np.max(w_vals)

    fontdict = {'fontsize': 10}
    fig, ax = plt.subplots(figsize=(9, 5.5))
    ax.set_ylim(0, 120)
    ax.set_xlim(min_w, max_w)
    ax.vlines((w_star,), (0,), (continuation_val,), lw=0.5)
    ax.set_yticks((0, 50, 100))
    ax.set_yticklabels((0, 50, 100), fontdict=fontdict)
    ax.set_xticks((min_w, w_star, max_w))
    ax.set_xticklabels((min_w, r"$w^*_1$", max_w), fontdict=fontdict)
    ax.plot(w_vals, w_vals + β * w_vals, alpha=0.8, linewidth=3,
            label=r"$w_1 + \beta w_1$")
    ax.plot(w_vals, [continuation_val]*(n+1), linewidth=3, alpha=0.8,
            label=r"$c + \beta \sum_{w'} \max\{c, w'\} \varphi(w')$" )
    ax.plot(w_vals, v, "k--", markersize=2, alpha=1.0, linewidth=2,
            label=r"$v_1(w_1)$")
    ax.legend(frameon=False, fontsize=fs, loc="upper left")
    if savefig:
        fig.savefig(figname)
    plt.show()

fig_v1(savefig=True)

In [3]:
from quantecon.distributions import BetaBinomial

import numpy as np
from numba import njit
from collections import namedtuple
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [4]:
φ = BetaBinomial(n=50, a=100, b=200).pdf()

In [5]:
φ

array([9.47465401e-09, 1.90254097e-07, 1.89832162e-06, 1.25427566e-05,
       6.17067935e-05, 2.40983918e-04, 7.77765720e-04, 2.13256868e-03,
       5.06815521e-03, 1.05989636e-02, 1.97361535e-02, 3.30312192e-02,
       5.00672786e-02, 6.91614209e-02, 8.75193404e-02, 1.01894858e-01,
       1.09542415e-01, 1.09072276e-01, 1.00844841e-01, 8.67601096e-02,
       6.95778357e-02, 5.20857161e-02, 3.64371566e-02, 2.38401316e-02,
       1.45968062e-02, 8.36624059e-03, 4.48909716e-03, 2.25461382e-03,
       1.05947988e-03, 4.65516517e-04, 1.91073370e-04, 7.31758378e-05,
       2.61087529e-05, 8.66281202e-06, 2.66702315e-06, 7.59880083e-07,
       1.99734835e-07, 4.82546790e-08, 1.06680218e-08, 2.14682757e-09,
       3.90773732e-10, 6.38444655e-11, 9.27411020e-12, 1.18361860e-12,
       1.30715161e-13, 1.22425907e-14, 9.45852331e-16, 5.78951642e-17,
       2.63322684e-18, 7.91385058e-20, 1.17916374e-21])

In [17]:
# NamedTuple Model
Model = namedtuple("Model", ("n", "w_vals", "φ", "β", "c"))


def create_job_search_model(
        n=50,        # wage grid size
        w_min=5.0,  # lowest wage
        w_max=55.0,  # highest wage
        a=200,       # wage distribution parameter
        b=100,       # wage distribution parameter
        β=0.96,      # discount factor
        c=10.0       # unemployment compensation
    ):
    """
    Creates the parameters for job search model and returns the
    instance of namedtuple Model
    """
    w_vals = np.linspace(w_min, w_max, n+1)
    φ = BetaBinomial(n, a, b).pdf()
    return Model(n=n, w_vals=w_vals, φ=φ, β=β, c=c)


In [21]:
@njit
def res_wage(model):
    """
    Computes reservation wage at t=1
    """
    β, c = model.β, model.c
    s = np.maximum(c, model.w_vals)
    h_1 = c + β * np.sum(s * model.φ)
    return h_1

In [22]:
job_model=create_job_search_model()

In [23]:
res_wage(job_model)

46.800000000012176